#Biblioteke

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from keras.utils import pad_sequences
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
import re
from nltk import tokenize
import nltk
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Importovanje podataka

In [ ]:
df=pd.read_csv('prediction_vijesti.csv',sep=',',header=None)
df.columns=['vijest','kategorija']
df.drop(index=df.index[0], axis=0, inplace=True)
df.head()

,vijest,kategorija
1,'Dva olovna sarkofaga otkrivena su u katedral...,2
2,'Američka svemirska agencija NASA saopćila je...,4
3,'Evropska komisija je predložila reformu cari...,3
4,'Administracija američkog predsjednika Joseph...,4
5,'Generalna direktorica Kliničkog centra Unive...,1


#Preprocesiranje

In [ ]:
samoglasnici = ['a', 'i', 'e', 'o', 'u']
suglasnici = ['b', 'c', 'č', 'ć', 'd', 'đ', 'dž', 'f', 'g', 'h', 'j', 'k', 'l', 'lj', 'm', 'n', 'nj', 'p', 'r', 's', 'š', 't', 'v', 'z', 'ž']

def stem(rijec):
  rijec = rijec.lower()
  duzina = len(rijec)

  #brojevi - osnovni
  if rijec.endswith(('jedan', 'dva', 'tri', 'cetiri', 'pet', 'sest', 'sedam', 'osam', 'devet', 'deset')) :
    return rijec

  #prilozi koji imaju 'nastavke'
  if rijec in ['daleko', 'teško', 'namjerno', 'mnogo', 'blizu', 'dolje', 'donekle', 'dotle', 'gore', 'gdje', 'odavdje', 'ovdje', 'ondje', 'tamo', 'ovamo', 'onamo',
               'kuda', 'kamo', 'naprijed', 'natrag', 'ikamo', 'nekamo', 'podalje', 'dalje', 'bliže', 'ponegdje', 'pozadi', 'sprijeda', 'straga', 'tu', 'vani', 'unutra',
               'sutra', 'kada', 'tada', 'ikada', 'odavno', 'odiskona', 'prije', 'maloprije', 'nekada', 'nikada', 'lani', 'preklani', 'brzo', 'sporo', 'kako', 'kojekako',
               'ovako', 'tako', 'nekako', 'nikako', 'svakojako', 'stoga', 'namjerno', 'milo', 'ovoliko', 'oniliko', 'premalo', 'premnogo', 'previše', 'više',
               'vrlo', 'malo', 'brže', 'dosta', 'puno', 'svuda', 'jednom', 'dvaput', 'triput', 'baš', 'među', 'samo', 'uskoro', 'tokom', 'nekoliko', 'među', 'između',
               'dakle'] :
               return rijec

  #prijedlozi:
  if rijec in ['poslije', 'u', 'na', 'iza', 'sa', 's', 'po', 'za', 'kroz', 'od', 'preko', 'prije', 'radi' ] :
    return rijec

  #glagoli svedeni na 3. lice jednine srednjeg roda jer infinitiv često ima drugačiji oblik (a radi lakšeg raspoznavanja)
  #proizvela, proizveo -> proizvelo
  #napisao, napisala -> napisalo

  #glagoli - 3. lice jednine srednjeg roda - proizv-elo, urad-ilo, opr-alo
  if duzina > 4 and rijec[-3] in samoglasnici and rijec[-2:] == 'lo':
    return rijec

  #glagoli - 3. lice jednine muškog roda - proizv-eo, urad-io, opr-ao
  if duzina > 3 and rijec[-2] in samoglasnici and rijec[-1] == 'o':
    rijec = rijec[:-1] + "lo"
    return rijec

  #glagoli - 3. lice jednine ženskog roda - proizv-ela, urad-ila, opr-ala, napis-ala
  #glagoli - 3. lice množine ženskog roda - proizv-ele, urad-ile, opr-ale, napis-ale
  #glagoli - 3. lice množine muškog i srednjeg roda - proizv-ela, urad-ili, opr-ali, napis-ali
  if duzina > 4 and rijec[-3] in samoglasnici and (rijec[-2:] in ['la', 'le', 'li']):
    rijec = rijec[:-2] + "lo"
    return rijec
 

  #imenice i brojevi - padezi
  #redoslijed ispitivanja bitan

  #americkim, drzavnim, poslovnim, pješačkih, poslovnih
  if(duzina > 4 and (rijec[-2:] == "im" or rijec[-2:] == "ih" or rijec[-2:] == "og")):
    rijec = rijec[:-2]

  #nastavak 'ima'
  #imenice muškog ili ženskog roda čiji dativ, instrumental ili lokativ množine završava na -ima - jelen-ima, momc-ima, televizor-ima, ljud-ima, kćer-ima
  elif(duzina > 4 and  rijec[-4] in suglasnici and rijec[-3:] == 'ima') :
    rijec = rijec[:-3]
    duzina = len(rijec)

    #umetak ev - panj-ev-ima, put-ev-ima
    if(duzina > 4 and rijec[-2:] == 'ev') :
      rijec = rijec[:-2]

    #umetak ov - rad-ov-ima, tip-ov-ima
    elif(duzina > 4 and rijec[-2:] == 'ov') :
      rijec = rijec[:-2]


  #nastavak 'ama'
  #imenice ženskog roda čiji dativ, instrumental ili lokativ množine završava na -ama - žen-ama, pilan-ama
  elif(duzina > 4 and rijec[-4] in suglasnici and rijec[-3:] == 'ama') :
    rijec = rijec[:-3]

  #nastavak 'iju'
  #imenice koje u genitivu imaju nastavak -iju -prst-iju, gost-iju
  elif(duzina > 4 and rijec[-4] in suglasnici and rijec[-3:] == 'iju') :
    rijec = rijec[:-3]


  #nastavak 'om'
  #imenice muškog roda čiji instrumental jednine završava na -om - jelen-om, momk-om, televizor-om, pacov-om
  elif(duzina > 3 and rijec[-3] in suglasnici and rijec[-2:] == 'om') :
    rijec = rijec[:-2]

  #nastavak 'em'
  #imenice muškog roda čiji instrumental jednine završava na -em - čitaoc-em
  elif(duzina > 3 and rijec[-3] in suglasnici and rijec[-2:] == 'em') :
    rijec = rijec[:-2]
    

  #nastavak 'a'
  #imenice muškog roda čiji genitiv (ili akuzativ) jednine završava na -a - slikar-a, grad-a, prostor-a, uspjeh-a, stol-a, kompjuter-a
  #imenice ženskog roda u nominativu jednine - djevojk-a, učiteljic-a, sreć-a, slog-a
  elif(duzina > 3 and rijec[-2] in suglasnici and rijec[-1] == 'a') :
    rijec = rijec[:-1]
    duzina = len(rijec)

    #umetak ev - panj-ev-a, put-ev-a
    if(duzina > 4 and rijec[-2:] == 'ev') :
      rijec = rijec[:-2]

    #umetak ov - rad-ov-a, tip-ov-a
    elif(duzina > 4 and rijec[-2:] == 'ov') :
      rijec = rijec[:-2]

  #nastavak 'e'
  #imenice muškog roda čiji genitiv (ili vokativ) jednine završava na -e - neznalic-e, kadij-e, arhitekt-e, Alij-e (jelen-e)
  #imenice ženskog roda čiji genitiv jednine završava na -e - Bosn-e, Fatim-e, učiteljic-e, vod-e
  #imenice srednjeg roda u nominativu jednine - čekanj-e, cvijeć-e, koplj-e, lic-e, proljeć-e
  elif(duzina > 3 and rijec[-2] in suglasnici and rijec[-1] == 'e') :
    rijec = rijec[:-1]

  #nastavak 'i'
  #imenice ženskog roda čiji genitiv jednine završava na -i - mladost-i, radost-i, čast-i, večer-i
  #imenice muškog roda čiji nominativ množine završava na -i - jelen-i, prozor-i, čistač-i, televizor-i
  elif(duzina > 3 and rijec[-2] in suglasnici and rijec[-1] == 'i') :
    rijec = rijec[:-1]
    duzina = len(rijec)

    #umetak ev - panj-ev-i, put-ev-i
    if(duzina > 4 and rijec[-2:] == 'ev') :
      rijec = rijec[:-2]

    #umetak ov - rad-ov-i, tip-ov-i
    elif(duzina > 4 and rijec[-2:] == 'ov') :
      rijec = rijec[:-2]

  #nastavak 'o'
  #imenice srednjeg roda u nominativu jednine - sel-o, čud-o, pism-o, jezer-o, neb-o, ok-o
  elif(duzina > 3 and rijec[-2] in suglasnici and rijec[-1] == 'o') :
    rijec = rijec[:-1]

  #nastavak 'u'
  #imenice muškog roda čiji dativ jednine završava na -u - prozor-u, jelen-u, televizor-u, pitanj-u
  elif(duzina > 3 and rijec[-2] in suglasnici and rijec[-1] == 'u') :
    rijec = rijec[:-1]

  #skim, škim, čkim nije izdvojeno jer riječi imaju različit tip tiječi samim tim i funkciju (imenica i pridjev)

  return rijec

In [ ]:
my_file = open("stopwords-bos.txt", "r")
  
data = my_file.read()
  
# replacing end of line('/n') with ' ' and
# splitting the text it further when '.' is seen.
data_into_list = data.replace('\n', '. ').split(".")
  
print(data_into_list)
stopwords = data_into_list
my_file.close()

['a', ' ako', ' ali', ' bi', ' bih', ' bila', ' bili', ' bilo', ' bio', ' bismo', ' biste', ' biti', ' da', ' do', ' duž', ' ga', ' hoće', ' hoćemo', ' hoćete', ' hoćeš', ' hoću', ' i', ' iako', ' ih', ' ili', ' iz', ' ja', ' je', ' jedna', ' jedne', ' jedno', ' jer', ' jesam', ' jesi', ' jesmo', ' jest', ' jeste', ' jesu', ' joj', ' još', ' ju', ' kada', ' kako', ' kao', ' koja', ' koje', ' koji', ' kojima', ' koju', ' kroz', ' li', ' me', ' mene', ' meni', ' mi', ' mimo', ' moj', ' moja', ' moje', ' mu', ' na', ' nad', ' nakon', ' nam', ' nama', ' nas', ' naš', ' naša', ' naše', ' našeg', ' ne', ' nego', ' neka', ' neki', ' nekog', ' neku', ' nema', ' neko', ' neće', ' nećemo', ' nećete', ' nećeš', ' neću', ' nešto', ' ni', ' nije', ' nikoga', ' nikoje', ' nikoju', ' nisam', ' nisi', ' nismo', ' niste', ' nisu', ' njega', ' njegov', ' njegova', ' njegovo', ' njemu', ' njezin', ' njezina', ' njezino', ' njen', ' njena', ' njeno', ' njih', ' njihov', ' njihova', ' njihovo', ' njim', ' 

In [ ]:
def remove_stopwords(text, stopwords):
  #print(text)
  #textwords = text.split()
  #print(textwords)
  textwords = text
  resultwords = [word for word in textwords if word.lower() not in stopwords]
  #print(resultwords)

  result = ' '.join(resultwords)
  #print(result)
  #print(stopwords)
  return result

In [ ]:
def tokeniziraj(text):
    
    text = text.replace('\\n', '').replace('\\t', '') #.replace('(\\\\\w+)+', '')
    
    text = re.sub(r'\d+\.', '', text) #remove 500., 4.4.
    text = re.sub(r'[0-9]', '', text)
    text = text.lower()

    sentences = re.split(r'[.!?]+', text)
    word_list = []

    for i in range(len(sentences)):
      index = sentences[i].find('izvor:')
      if (index == -1):
        sentences[i] = re.sub(r'[^\w\s]','', sentences[i])
        words = sentences[i].strip().split()
        word_list.extend(words)

    return word_list

In [ ]:
def preprocesiraj(text, bez_stopriječi, stemming):
  
  
  #segmentacija na rečenice
  text = text.split("\\n")

  #tokenizacija sa dodatnim preprocesiranjem popu izbacivanja interpunkcije i sl
  text_list = []
  for sentence in text:
    text_list = text_list + tokeniziraj(sentence)
  
  #stop riječi
  if bez_stopriječi:
    text_list = remove_stopwords(text_list, stopwords) 
 
  
  if stemming:
    text_list = text_list.split()
    stem_text_list = ''
    for word in text_list:
      stem_text_list = stem_text_list + " " + stem(word)
    text_list = stem_text_list

  return text_list


In [ ]:
df['vijest']=df['vijest'].apply(lambda x:preprocesiraj(x, True, True))

In [ ]:
df.head()

,vijest,kategorija
1,dva olovn sarkofag otkriven su u katedralo no...,2
2,američk svemirsk agencij nas saopćilo je da j...,4
3,evropsk komisij je predložilo reform carinsk ...,3
4,administracij američk predsjednik joseph bide...,4
5,generaln direktoric kliničk centr univerzitet...,1


In [ ]:
df.iloc[0,0]

' dva olovn sarkofag otkriven su u katedralo notr dam u pariz koj se rekonstruir nakon razorn požar godin francusk arheoloz su potvrdilo da je riječ o izvanredn otkrić prenilo je tanjug pisanj britansk dnevnik guardian prv sarkofag sadrž posmrtn ostatk prvosveštenik koj je umr godin ali čovjek koj je sahranjen u drug sarkofag još uvijek nij identificiran naučnic ukazuj da možd nikada neć bit utvrđen identitet osob sahranjen u drug sarkofag ali na osnov procjen stručnjak riječ je o mlad bogat i privilegovan čovjek plemic i vitez koj je po svem sudeć živilo u stoljeć izvanredan naučn značaj grobnic su otkriven u dijel koj se koristilo kao skladišt za skulptur i fragment originaln krovn pokrivač katedralo iz stoljeć zakopan ispod pod u poprečn brod crkv koj je sazidan tako da razdvaj svetilišt od ostatk crkv sarkofaz su otkriven na mjest gdje je trebalo da bud postavljen skelo za podizanj nov tornj katedralo naučnic su uvjeren da je riječ o otkrić od izvanredn naučn značaj dodaj se da su 

#EMBEDDINGS

Korištenje CountVectorizer-a

In [ ]:
vijest_text = df['vijest'].tolist()

vectorizer = CountVectorizer()

vectorizer.fit(vijest_text)

X_count = vectorizer.transform(vijest_text)

In [ ]:
X_count.shape

(500, 19162)

Korištenje naših embeddinga

In [ ]:
def embedding(word, model):
  try:
    return model.wv[word]
  except:
    return 0

In [ ]:
def primijeni_embeddings(vijest, model):
  return [embedding(word, model) for word in vijest.split() if word.strip() != '']

Korištenje našeg word2vec embeddinga za bosanski jezik

In [ ]:
with open('word2vec_model.pkl', 'rb') as f:
    model1 = pickle.load(f)

Average za doc embedding

In [ ]:
def create_e_document_embedding(document, model):
 
    words = document.split()

    word_embeddings = [model.wv[word] for word in words if word in model.wv]

    if len(word_embeddings) > 0:
        e_document_embedding = np.mean(word_embeddings, axis=0)
        return e_document_embedding
    else:
        return None

# Example usage
document = "dva olovn sarkofag otkriven"
e_document_embedding = create_e_document_embedding(document, model1)


In [ ]:
e_document_embedding

array([-0.2563055 ,  0.4289508 ,  0.13968325, -0.00567187, -0.08088234,
       -0.53153604, -0.07222753,  0.6712473 , -0.31782824, -0.3947413 ,
       -0.00895776, -0.73699784,  0.2125389 ,  0.22470176,  0.10303932,
       -0.02500728,  0.4065431 , -0.68732864,  0.21216793, -0.9122593 ,
        0.02473907, -0.10761615,  0.4413371 , -0.2624526 , -0.08880218,
       -0.00385906, -0.12002441, -0.14038445, -0.10201816, -0.04170978,
        0.3545941 , -0.126835  , -0.02580099, -0.4054303 , -0.3405647 ,
        0.01246467,  0.54517835, -0.01095219,  0.0547645 , -0.07435878,
       -0.07855097,  0.10011658, -0.03414258, -0.22590472,  0.25123808,
       -0.37678325, -0.37836537, -0.10957455, -0.07296759,  0.20370527,
        0.23450433, -0.11770424, -0.12544435,  0.0753583 ,  0.04574575,
       -0.13959277,  0.2105022 , -0.10219141,  0.03999852,  0.3539399 ,
       -0.10132361, -0.14637238,  0.08807174,  0.0704119 , -0.36593667,
        0.2636588 ,  0.17397358,  0.09434251, -0.27900502, -0.03

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

def create_e_document_weighted_embedding(document, model):
    
    words = document.split()

    tfidf_scores = tfidf_vectorizer.fit_transform([document]).toarray()[0]

    word_embeddings = []
    weighted_scores = []

    for word, score in zip(words, tfidf_scores):
        if word in model.wv:
            word_embedding = model.wv[word]
            
            word_embeddings.append(word_embedding)
            weighted_scores.append(score)

    if len(word_embeddings) > 0:
        word_embeddings = np.array(word_embeddings)
        weighted_scores = np.array(weighted_scores)

        weighted_average = np.average(word_embeddings, axis=0, weights=weighted_scores)

        return weighted_average
    else:
        return None


document = "dva olovn sarkofag otkriven"
e_document_embedding = create_e_document_weighted_embedding(document, model1)


In [ ]:
df['vijest'].head()

1     dva olovn sarkofag otkriven su u katedralo no...
2     američk svemirsk agencij nas saopćilo je da j...
3     evropsk komisij je predložilo reform carinsk ...
4     administracij američk predsjednik joseph bide...
5     generaln direktoric kliničk centr univerzitet...
Name: vijest, dtype: object

*   word2vec_avg je embedding za dokument (vijest) napravljen računanjem prosjeka svih w2v embeddinga za pojedine riječi u dokumentu
*   word2vec_weight_avg umjesto prosjeka korišten weighted average, prvo izračunat tfidf score za riječi u dokumentu


In [ ]:
#ovdje svaka rijec ima svoj embedding
df['word2vec_avg']=df['vijest'].apply(lambda x:create_e_document_embedding(x, model1))
df['word2vec_weight_avg']=df['vijest'].apply(lambda x:create_e_document_weighted_embedding(x, model1))

df.head()

,vijest,kategorija,word2vec_avg,word2vec_weight_avg
1,dva olovn sarkofag otkriven su u katedralo no...,2,"[-0.34594575, 0.2839031, 0.17603719, 0.3962955...","[-0.3476191422114858, 0.24894839840098504, 0.1..."
2,američk svemirsk agencij nas saopćilo je da j...,4,"[-0.38080397, 0.22650556, 0.15119454, 0.408468...","[-0.36843028626715124, 0.175907754461902, 0.13..."
3,evropsk komisij je predložilo reform carinsk ...,3,"[-0.32226032, 0.21427296, 0.14981331, 0.364215...","[-0.3436872317729629, 0.2293367771395281, 0.10..."
4,administracij američk predsjednik joseph bide...,4,"[-0.42571414, 0.16877402, 0.1836608, 0.3385192...","[-0.4412821094911702, 0.16989303872241127, 0.1..."
5,generaln direktoric kliničk centr univerzitet...,1,"[-0.3985443, 0.35517603, 0.28379977, 0.3238662...","[-0.4764896186191194, 0.38053628077828555, 0.3..."


In [ ]:
X=df['word2vec_avg'].to_numpy()
X=X.reshape(-1,1)
X.shape

(500, 1)

In [ ]:
X2=df['word2vec_weight_avg'].to_numpy()
X2=X2.reshape(-1,1)
X2.shape

(500, 1)

In [ ]:
X_w2v_avg=np.concatenate(np.concatenate(X,axis=0),axis=0).reshape(-1,100)
X_w2v_avg.shape

(500, 100)

In [ ]:
X_w2v_weight_avg=np.concatenate(np.concatenate(X2,axis=0),axis=0).reshape(-1,100)
X_w2v_weight_avg.shape

(500, 100)

Korištenje našeg FastText embeddinga za bosanski jezik

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
with open('fasttext_model.pkl', 'rb') as f:
    model2 = pickle.load(f)

In [ ]:
df['fasttext_avg']=df['vijest'].apply(lambda x:create_e_document_embedding(x, model2))
df['fasttext_weight_avg']=df['vijest'].apply(lambda x:create_e_document_weighted_embedding(x, model2))
df.head()

,vijest,kategorija,word2vec_avg,word2vec_weight_avg,fasttext_avg,fasttext_weight_avg
1,dva olovn sarkofag otkriven su u katedralo no...,2,"[-0.34594575, 0.2839031, 0.17603719, 0.3962955...","[-0.3476191422114858, 0.24894839840098504, 0.1...","[-0.19047846, 0.049747154, -0.1451849, -0.1318...","[-0.17449348821733768, 0.06817948322460644, -0..."
2,američk svemirsk agencij nas saopćilo je da j...,4,"[-0.38080397, 0.22650556, 0.15119454, 0.408468...","[-0.36843028626715124, 0.175907754461902, 0.13...","[-0.24016836, 0.056401964, -0.09147829, -0.161...","[-0.28371308767236775, 0.0554799237778331, -0...."
3,evropsk komisij je predložilo reform carinsk ...,3,"[-0.32226032, 0.21427296, 0.14981331, 0.364215...","[-0.3436872317729629, 0.2293367771395281, 0.10...","[-0.23297836, 0.012733971, -0.15380538, -0.161...","[-0.20176888890814224, -0.0023879029905607293,..."
4,administracij američk predsjednik joseph bide...,4,"[-0.42571414, 0.16877402, 0.1836608, 0.3385192...","[-0.4412821094911702, 0.16989303872241127, 0.1...","[-0.30245286, 0.006994489, -0.09531504, -0.193...","[-0.2945003990632174, 0.030109189739771967, -0..."
5,generaln direktoric kliničk centr univerzitet...,1,"[-0.3985443, 0.35517603, 0.28379977, 0.3238662...","[-0.4764896186191194, 0.38053628077828555, 0.3...","[-0.19063023, 0.112900995, -0.1254667, -0.1578...","[-0.2045042071923879, 0.08948224389522294, -0...."


In [ ]:
X3=df['fasttext_avg'].to_numpy()
X3=X3.reshape(-1,1)
X3.shape

(500, 1)

In [ ]:
X_ft_avg=np.concatenate(np.concatenate(X3,axis=0),axis=0).reshape(-1,100)
X_ft_avg.shape

(500, 100)

In [ ]:
X4=df['fasttext_weight_avg'].to_numpy()
X4=X4.reshape(-1,1)
X4.shape

(500, 1)

In [ ]:
X_ft_weighted_avg=np.concatenate(np.concatenate(X4,axis=0),axis=0).reshape(-1,100)
X_ft_weighted_avg.shape

(500, 100)

In [ ]:
df.head()

,vijest,kategorija,word2vec_avg,word2vec_weight_avg,fasttext_avg,fasttext_weight_avg
1,dva olovn sarkofag otkriven su u katedralo no...,2,"[-0.34594575, 0.2839031, 0.17603719, 0.3962955...","[-0.3476191422114858, 0.24894839840098504, 0.1...","[-0.19047846, 0.049747154, -0.1451849, -0.1318...","[-0.17449348821733768, 0.06817948322460644, -0..."
2,američk svemirsk agencij nas saopćilo je da j...,4,"[-0.38080397, 0.22650556, 0.15119454, 0.408468...","[-0.36843028626715124, 0.175907754461902, 0.13...","[-0.24016836, 0.056401964, -0.09147829, -0.161...","[-0.28371308767236775, 0.0554799237778331, -0...."
3,evropsk komisij je predložilo reform carinsk ...,3,"[-0.32226032, 0.21427296, 0.14981331, 0.364215...","[-0.3436872317729629, 0.2293367771395281, 0.10...","[-0.23297836, 0.012733971, -0.15380538, -0.161...","[-0.20176888890814224, -0.0023879029905607293,..."
4,administracij američk predsjednik joseph bide...,4,"[-0.42571414, 0.16877402, 0.1836608, 0.3385192...","[-0.4412821094911702, 0.16989303872241127, 0.1...","[-0.30245286, 0.006994489, -0.09531504, -0.193...","[-0.2945003990632174, 0.030109189739771967, -0..."
5,generaln direktoric kliničk centr univerzitet...,1,"[-0.3985443, 0.35517603, 0.28379977, 0.3238662...","[-0.4764896186191194, 0.38053628077828555, 0.3...","[-0.19063023, 0.112900995, -0.1254667, -0.1578...","[-0.2045042071923879, 0.08948224389522294, -0...."


#Klasifikacija - Train test podjela, fit, predict, report

In [ ]:
def classification(podaci_u_embedding_obliku, labele):
  #y=df['kategorija']
  X_train,X_test,y_train,y_test=train_test_split(podaci_u_embedding_obliku,labele,test_size=0.2,random_state = 0)
  X_train.shape,X_test.shape

  #SVM
  clf=LinearSVC()
  clf.fit(X_train,y_train)

  y_pred=clf.predict(X_test)
  confusion_matrix(y_test,y_pred)

  print(classification_report(y_test,y_pred))

  return y_pred, y_test


In [ ]:
classification(X_w2v_avg, df['kategorija'])

              precision    recall  f1-score   support

           1       0.77      0.74      0.76        23
           2       0.94      1.00      0.97        17
           3       0.80      0.83      0.82        24
           4       0.67      0.80      0.73        20
           5       0.64      0.44      0.52        16

    accuracy                           0.77       100
   macro avg       0.76      0.76      0.76       100
weighted avg       0.77      0.77      0.76       100



(array(['2', '3', '5', '4', '5', '3', '5', '2', '1', '2', '1', '4', '5',
        '2', '4', '1', '1', '4', '1', '4', '3', '4', '3', '2', '3', '3',
        '2', '4', '2', '4', '1', '5', '1', '4', '3', '4', '3', '4', '1',
        '2', '3', '2', '1', '4', '1', '4', '3', '3', '2', '5', '3', '3',
        '1', '3', '5', '5', '1', '3', '1', '1', '2', '1', '3', '1', '4',
        '3', '3', '2', '4', '1', '4', '3', '3', '2', '1', '1', '5', '2',
        '3', '5', '2', '5', '2', '3', '1', '4', '4', '1', '2', '4', '1',
        '2', '3', '4', '4', '4', '3', '3', '4', '4'], dtype=object),
 91     2
 255    4
 284    5
 446    3
 462    5
       ..
 373    1
 57     3
 441    3
 61     4
 209    4
 Name: kategorija, Length: 100, dtype: object)

In [ ]:
classification(X_w2v_weight_avg, df['kategorija'])

              precision    recall  f1-score   support

           1       0.79      0.65      0.71        23
           2       0.77      1.00      0.87        17
           3       0.76      0.67      0.71        24
           4       0.48      0.55      0.51        20
           5       0.47      0.44      0.45        16

    accuracy                           0.66       100
   macro avg       0.65      0.66      0.65       100
weighted avg       0.67      0.66      0.66       100



(array(['2', '3', '5', '4', '4', '4', '5', '2', '1', '2', '1', '4', '5',
        '2', '2', '1', '2', '4', '1', '4', '3', '4', '3', '2', '3', '3',
        '2', '4', '2', '5', '1', '5', '1', '4', '3', '4', '3', '4', '5',
        '2', '3', '2', '1', '4', '1', '4', '3', '3', '2', '5', '3', '3',
        '1', '3', '5', '5', '4', '3', '1', '1', '2', '5', '3', '1', '4',
        '3', '3', '2', '4', '1', '4', '3', '3', '2', '1', '1', '5', '2',
        '4', '5', '2', '5', '2', '5', '1', '2', '5', '1', '2', '4', '1',
        '2', '3', '4', '4', '4', '4', '3', '2', '1'], dtype=object),
 91     2
 255    4
 284    5
 446    3
 462    5
       ..
 373    1
 57     3
 441    3
 61     4
 209    4
 Name: kategorija, Length: 100, dtype: object)

In [ ]:
classification(X_count, df['kategorija'])

              precision    recall  f1-score   support

           1       0.61      0.87      0.71        23
           2       1.00      0.88      0.94        17
           3       0.94      0.71      0.81        24
           4       0.89      0.80      0.84        20
           5       0.69      0.69      0.69        16

    accuracy                           0.79       100
   macro avg       0.83      0.79      0.80       100
weighted avg       0.82      0.79      0.80       100



(array(['2', '5', '5', '4', '5', '3', '5', '2', '5', '2', '1', '4', '5',
        '2', '4', '1', '1', '4', '1', '3', '1', '1', '3', '2', '3', '3',
        '2', '4', '2', '4', '5', '5', '1', '4', '3', '5', '3', '5', '1',
        '2', '3', '2', '1', '1', '1', '4', '3', '3', '2', '4', '3', '1',
        '1', '3', '5', '1', '4', '1', '1', '1', '2', '1', '1', '1', '1',
        '3', '3', '2', '4', '1', '4', '3', '1', '1', '1', '1', '5', '4',
        '1', '5', '2', '1', '1', '3', '1', '4', '4', '1', '2', '5', '1',
        '2', '5', '4', '4', '5', '3', '3', '4', '1'], dtype=object),
 91     2
 255    4
 284    5
 446    3
 462    5
       ..
 373    1
 57     3
 441    3
 61     4
 209    4
 Name: kategorija, Length: 100, dtype: object)

In [ ]:
classification(X_ft_avg, df['kategorija'])

              precision    recall  f1-score   support

           1       0.95      0.78      0.86        23
           2       1.00      0.94      0.97        17
           3       0.87      0.83      0.85        24
           4       0.76      0.95      0.84        20
           5       0.88      0.94      0.91        16

    accuracy                           0.88       100
   macro avg       0.89      0.89      0.89       100
weighted avg       0.89      0.88      0.88       100



(array(['2', '3', '5', '4', '5', '3', '5', '5', '5', '2', '1', '4', '5',
        '2', '4', '1', '1', '4', '1', '4', '3', '4', '3', '2', '3', '3',
        '2', '4', '2', '4', '1', '5', '1', '4', '3', '3', '3', '5', '1',
        '2', '3', '2', '1', '4', '1', '4', '3', '3', '2', '4', '3', '3',
        '1', '3', '5', '5', '4', '3', '1', '1', '2', '5', '5', '5', '4',
        '3', '4', '2', '4', '1', '4', '3', '3', '2', '1', '1', '4', '5',
        '3', '5', '2', '1', '2', '3', '1', '4', '4', '1', '2', '5', '1',
        '2', '5', '4', '4', '4', '3', '3', '4', '4'], dtype=object),
 91     2
 255    4
 284    5
 446    3
 462    5
       ..
 373    1
 57     3
 441    3
 61     4
 209    4
 Name: kategorija, Length: 100, dtype: object)

In [ ]:
classification(X_ft_weighted_avg, df['kategorija'])

              precision    recall  f1-score   support

           1       0.81      0.74      0.77        23
           2       1.00      0.94      0.97        17
           3       0.86      0.79      0.83        24
           4       0.73      0.95      0.83        20
           5       0.87      0.81      0.84        16

    accuracy                           0.84       100
   macro avg       0.85      0.85      0.85       100
weighted avg       0.85      0.84      0.84       100



(array(['2', '3', '5', '4', '5', '3', '5', '5', '5', '2', '4', '4', '5',
        '2', '4', '1', '1', '4', '1', '4', '3', '4', '3', '2', '3', '3',
        '2', '4', '2', '4', '5', '5', '1', '4', '1', '3', '3', '5', '1',
        '2', '3', '2', '1', '4', '1', '4', '3', '3', '2', '4', '3', '3',
        '1', '3', '5', '1', '4', '3', '1', '1', '2', '5', '5', '1', '4',
        '3', '4', '2', '4', '1', '4', '3', '3', '2', '1', '1', '4', '5',
        '3', '5', '2', '1', '2', '3', '1', '4', '4', '1', '2', '5', '1',
        '2', '1', '4', '4', '4', '3', '3', '4', '4'], dtype=object),
 91     2
 255    4
 284    5
 446    3
 462    5
       ..
 373    1
 57     3
 441    3
 61     4
 209    4
 Name: kategorija, Length: 100, dtype: object)

# Klastering - treniranje, report

In [ ]:
def clustering(podaci_u_embedding_obliku, labele):

  labele = labele.astype(int)
  kmeans_model = KMeans(n_clusters = 5)
  kmeans_model.fit(podaci_u_embedding_obliku)

  clusters = kmeans_model.labels_
  clusters = [x+1 for x in clusters]

  confusion_matrix(labele,clusters)

  print(classification_report(labele,clusters))

  return 

In [ ]:
clustering(X_count, df['kategorija'])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


              precision    recall  f1-score   support

           1       0.26      0.66      0.37       110
           2       0.27      0.09      0.13        90
           3       0.00      0.00      0.00       110
           4       0.18      0.29      0.22       100
           5       0.00      0.00      0.00        90

    accuracy                           0.22       500
   macro avg       0.14      0.21      0.15       500
weighted avg       0.14      0.22      0.15       500



In [ ]:
clustering(X_w2v_avg, df['kategorija'])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


              precision    recall  f1-score   support

           1       0.43      0.61      0.50       110
           2       0.97      0.39      0.56        90
           3       0.80      0.48      0.60       110
           4       0.27      0.26      0.27       100
           5       0.18      0.29      0.22        90

    accuracy                           0.41       500
   macro avg       0.53      0.41      0.43       500
weighted avg       0.53      0.41      0.44       500



In [ ]:
clustering(X_w2v_weight_avg, df['kategorija'])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


              precision    recall  f1-score   support

           1       0.46      0.58      0.51       110
           2       0.11      0.18      0.14        90
           3       0.77      0.51      0.61       110
           4       0.40      0.51      0.45       100
           5       0.00      0.00      0.00        90

    accuracy                           0.37       500
   macro avg       0.35      0.36      0.34       500
weighted avg       0.37      0.37      0.36       500



In [ ]:
clustering(X_ft_avg, df['kategorija'])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


              precision    recall  f1-score   support

           1       0.03      0.02      0.02       110
           2       0.00      0.00      0.00        90
           3       0.22      0.31      0.26       110
           4       0.76      0.51      0.61       100
           5       0.37      0.53      0.43        90

    accuracy                           0.27       500
   macro avg       0.27      0.27      0.26       500
weighted avg       0.27      0.27      0.26       500



In [ ]:
clustering(X_ft_weighted_avg, df['kategorija'])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


              precision    recall  f1-score   support

           1       0.64      0.40      0.49       110
           2       0.89      0.84      0.87        90
           3       0.88      0.60      0.71       110
           4       0.02      0.02      0.02       100
           5       0.16      0.32      0.21        90

    accuracy                           0.43       500
   macro avg       0.52      0.44      0.46       500
weighted avg       0.53      0.43      0.46       500

